In [ ]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from threading import Thread

import requests
from bs4 import BeautifulSoup

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#List of all the cities available on zameen.pk
cities = ['Islamabad-3', 'Karachi-2', 'Lahore-1', 'Rawalpindi-41', 'Abbottabad-385', 'Abdul_Hakim-10594', 'Ahmedpur_East-12360',
          'Alipur-10242', 'Arifwala-9517', 'Astore-1763', 'Attock-1233', 'Awaran-1761', 'Badin-8857', 'Bagh-966', 
          'Bahawalnagar-557', 'Bahawalpur-23', 'Bannu-1735', 'Barnala-13977', 'Bhakkar-1720', 'Bhalwal-11142', 'Bhimber-1749', 
          'Buner-11286', 'Burewala-1059', 'Chaghi-1747', 'Chakwal-751', 'Choa_Saidan_Shah-14284', 'Charsadda-11420', 
          'Chichawatni-8860', 'Chiniot-1469', 'Chishtian_Sharif-10512', 'Chitral-1731', 'Chunian-1061', 'Dadu-1727', 
          'Daharki-11967', 'Daska-1509', 'Depalpur-9178', 'Dera_Ghazi_Khan-26', 'Dera_Ismail_Khan-8244', 'Dijkot-10645', 
          'Dina-12718', 'Duniya_Pur-8474', 'Faisalabad-16', 'FATA-1737', 'Fateh_Jang-1293', 'Gaarho-10894', 'Gadoon-11915', 
          'Galyat-8119', 'Gharo-636', 'Ghotki-8810', 'Gilgit-1753', 'Gojra-10281', 'Gujar_Khan-8338', 'Gujranwala-327', 
          'Ghakhar-13272', 'Gujrat-20', 'Gwadar-389', 'Hafizabad-1714', 'Hangu-11739', 'Harappa-11634', 'Haripur-1048', 
          'Haroonabad-1152', 'Hasilpur-9687', 'Hassan_Abdal-399', 'Haveli_Lakha-10402', 'Hazro-12823', 'Hub_Chowki-9844', 
          'Hunza-1546', 'Hyderabad-30', 'Jacobabad-32', 'Jahanian-11126', 'Jalalpur_Jattan-11026', 'Jampur-10484', 'Jamshoro-1178', 
          'Jauharabad-8511', 'Jhang-1142', 'Jhelum-19', 'Kabirwala-9873', 'Kaghan-9202', 'Kahror_Pakka-10279', 'Kalat-1750', 
          'Kamalia-10416', 'Kamoki-10346', 'Karak-9484', 'Kasur-544', 'Khairpur-8806', 'Khanewal-1685', 'Khanpur-10168', 
          'Kharian-1305', 'Khipro-12390', 'Khushab-8510', 'Khuzdar-1757', 'Kohat-1430', 'Kot_Addu-9749', 'Kotli-968', 'Kotri-8591', 
          'Lakki_Marwat-10205', 'Lalamusa-9837', 'Larkana-586', 'Lasbela-548', 'Layyah-1661', 'Liaquatpur-11406', 'Lodhran-9872', 
          'Loralai-1742', 'Lower_Dir-10482', 'Mailsi-9422', 'Makran-1767', 'Malakand-1384', 'Mandi_Bahauddin-1496', 'Mangla-14350', 
          'Mansehra-771', 'Mardan-440', 'Matiari-8606', 'Mian_Channu-9636', 'Mianwali-8310', 'Mirpur-1349', 'Mirpur_Khas-1558', 
          'Mirpur_Sakro-10893', 'Multan-15', 'Muridke-8116', 'Murree-36', 'Muzaffarabad-977', 'Muzaffargarh-1722', 
          'Nankana_Sahib_-1687', 'Naran-1258', 'Narowal-541', 'Nasirabad-1752', 'Naushahro_Feroze-8801', 'Nawabshah-1704', 
          'Neelum-1741', 'Nowshera-1424', 'Okara-470', 'Others-979', 'Others_Azad_Kashmir-1528', 'Others_Balochistan-1529', 
          'Others_Gilgit_Baltistan-1530', 'Others_Khyber_Pakhtunkhwa-1531', 'Others_Punjab-1532', 'Others_Sindh-1533', 
          'Pakpattan-1716', 'Peshawar-17', 'Pind_Dadan_Khan-10678', 'Pindi_Bhattian-975', 'Pir_Mahal-9508', 'Quetta-18', 
          'Rahim_Yar_Khan-40', 'Rajanpur-9645', 'Ratwal-9027', 'Rawalkot-976', 'Rohri-1725', 'Pasrur-17168', 'Sadiqabad-9538', 
          'Sahiwal-782', 'Samundri-10632', 'Sanghar-8609', 'Sangla_Hill-8563', 'Sarai_Alamgir-1034', 'Sargodha-778', 'Sehwan-8607', 
          'Shabqadar-13211', 'Shahdadpur-9029', 'Shahkot-8552', 'Shakargarh-12170', 'Sheikhupura-44', 'Shikarpur-8808', 
          'Shorkot-10334', 'Sialkot-480', 'Sibi-1744', 'Skardu-1545', 'Sudhnoti-1745', 'Sujawal-14329', 'Sukkur-45', 'Swabi-3094', 
          'Swat-1506', 'Talagang-12137', 'Tando_Adam-9028', 'Tando_Allahyar-11315', 'Tando_Bago-11700', 'Tando_Muhammad_Khan-13166', 
          'Taxila-464', 'Tharparkar-12439', 'Thatta-1729', 'Toba_Tek_Singh-1658', 'Turbat-12271', 'Vehari-1432', 'Wah-459', 
          'Wazirabad-1395', 'Waziristan-1765', 'Yazman-12504', 'Zhob-1739']

In [ ]:
# Web Scraping from zameen.pk

# General Format for Buying
# https://www.zameen.com/{type}/{city}-{page}.html

# Format for Buy, Homes, Lahore 
# https://www.zameen.com/Homes/Lahore-1-1.html

# Format for Buy, Plots, Lahore
# https://www.zameen.com/Plots/Lahore-1-1.html

# Format for Buy, Commercial, Lahore
# https://www.zameen.com/Commercial/Lahore-1-1.html


# Format for Rents

# Format for Rent, Homes, Lahore
# https://www.zameen.com/Rentals/Lahore-1-1.html
# https://www.zameen.com/Rentals/{city}-{page}.html

# Format for Rent, Plots, Lahore
# https://www.zameen.com/Rentals_Plots/Lahore-1-1.html
# https://www.zameen.com/Rentals_Plots/{city}-{page}.html

# Format for Rent, Commercial, Lahore
# https://www.zameen.com/Rentals_Commercial/Lahore-1-1.html
# https://www.zameen.com/Rentals_Commercial/{city}-{page}.html

#Function to Extract Data
def extract_data(city, p_type, p_purpose):
            #Initialize variables
            data1 = []
            basic_url = ''
            page = 0
            
            # Set the value of basic url
            if p_purpose == 'buy':
                basic_url = 'https://www.zameen.com/' + p_type + '/'      
            elif p_purpose == 'rent':
                
                if p_type == 'Homes':
                    basic_url = 'https://www.zameen.com/Rentals/'
                elif p_type == 'Plots':
                    basic_url = 'https://www.zameen.com/Rentals_Plots/'
                elif p_type == 'Commercial':
                    basic_url = 'https://www.zameen.com/Rentals_Commercial/'
                
            #Loop until data from all pages is extracted
            while(True):
                # Create the url for a specific page
                url = basic_url + city + '-' + str(page) + '.html'

                # Fetch the HTML content of the page
                response = requests.get(url)
                # Break if the page does not exist
                if(not response): 
                    break

                # Parse the HTML content
                soup = BeautifulSoup(response.text, 'html.parser')
                    
                # Find all the links to properties on the given page
                link_elements = soup.find_all(attrs = {"aria-label": "Listing link"})
                
                # Extract data for each property
                for element in link_elements:
                    link = "https://www.zameen.com/" + element['href']
                    response = requests.get(link)
                        
                    # Parse the HTML content
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # Get all the required features  
                    title = soup.find('h1', class_ = '_64bb5b3b').text
                    type1 = soup.find(attrs = {"aria-label": "Type"}).text
                    price = soup.find(attrs = {"aria-label": "Price"}).text
                    location = soup.find(attrs = {"aria-label": "Location"}).text
                    baths = soup.find(attrs = {"aria-label": "Baths"}).text
                    beds = soup.find(attrs = {"aria-label": "Beds"}).text
                    area = soup.find(attrs = {"aria-label": "Area"}).text
                    purpose = soup.find(attrs = {"aria-label": "Purpose"}).text
                    added = soup.find(attrs = {"aria-label": "Creation date"}).text
                    description = soup.find(attrs = {"aria-label": "Property description"}).text

                    #Append into the array
                    data1.append([link, title, type1, price, location, baths, beds, area, purpose, added, description])

                # Print the page number and (city, p_type, purpose) of the above data
                print(page, '(', city, p_type, p_purpose , ')')

                #increment page number
                page += 1

            # Append the data1 array into the main data array
            data.extend(data1)


In [ ]:

#List of purposes and types of properties
purposes = ['buy', 'rent']
types = ['Homes', 'Plots', 'Commercial']

#initialize the arrays for data and threads
data = []
threads = []

#Loop through all cities (Use few cities at a time in order to avoid hundreds of threads executing at a time)
for city in cities:
    # Loop through all the property types
    for p_type in types: 
        # Loop through all the purposes i.e., sale and rent
        for p_purpose in purposes:
          
          #Create threads to extract data from multiple pages at a time
          t = Thread(target=extract_data, args=(city, p_type, p_purpose,))
          # Start the thread
          t.start()  
          # Append into the threads array
          threads.append(t)

# Wait for all the threads to complete   
for t in threads:
    t.join()            
                    
# Convert the list to a dataframe
data_df = pd.DataFrame(data, columns=["Url", "Title", "Type", "Price", "Location", "Baths", "Beds", "Area", "Purpose", "Date_Added", "Description"])
            
# # Convert the dataframe to a CSV file
data_df.to_csv("/content/drive/MyDrive/DWDM Data/RealEstateData.csv", index=False)


0 ( Karachi-2 Plots buy )
0 (0 0 0 (00000 ( Rawalpindi-41 Commercial rent )( Islamabad-3 Homes buy )
( Lahore-1 000 (0 ( Abbottabad-385  Lahore-1 Plots buy )
 ( 00 ( Lahore-1 Commercial rent )

 ( Rawalpindi-41 0 ( Karachi-2 Homes Karachi-2 Plots rent )0 ( Abbottabad-385 Commercial Plots buy )
 ( Islamabad-3 Commercial buy )
 buy )
0 buy
0 Homes buy )
( Abbottabad-385 Homes buy )
 0 )
( Islamabad-3  0  Karachi-2 Homes rent )
Plots buy )
( Rawalpindi-41 Commercial buy )
00Plots buy )
 ( Islamabad-3 Commercial rent )
(Lahore-1 0( Abbottabad-385 Plots rent )
 ( Abdul_Hakim-10594 Commercial buy )
( Abdul_Hakim-10594 Plots buy )
0 ( Abbottabad-385 Homes rent )
 Homes rent )
 Islamabad-3 Plots rent )
 ( Rawalpindi-41 Homes rent  0 ( ( Lahore-1 Commercial buy )
0( Abbottabad-385 Commercial rent )
)
 Karachi-2 Commercial buy )
00 ( Abdul_Hakim-10594 Commercial rent )
 ( Rawalpindi-41 Plots rent )
 ( Abdul_Hakim-10594 Homes rent )
00 ( Ahmedpur_East-12360 Homes rent )
0 ( Rawalpindi-41 Homes bu

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



0 ( Barnala-13977 Plots rent )



KeyboardInterrupt



1 ( Abdul_Hakim-10594 Plots rent )
1 ( Ahmedpur_East-12360 Plots rent )
1 ( Alipur-10242 Plots rent )
